### LeitlinienGPT - Ingestion Pipeline

#### 1. Setup Environment **Keys**

In [6]:
#import utils
import os
import openai
import param
from llama_index.core import SimpleDirectoryReader, Document
from huggingface_hub import login
from llama_parse import LlamaParse
from dotenv import load_dotenv

# Loading Environment variables:
dotenv_path = 'KEYs.env'  
_ = load_dotenv(dotenv_path)

# Loading Environment variables:
openai.api_key = os.environ['OPENAI_API_KEY']
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']
LLAMA_CLOUD_API_KEY = os.environ['OPENAI_API_KEY']
hf_token = os.environ['hf_token']
login(token=hf_token)


/Users/timstrohmeyer/Desktop/Official LGPT repo/DEV/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/timstrohmeyer/.cache/huggingface/token
Login successful


#### 2. Parse Documents

We have 10 API keys:
Hence, Parse the documents as follows:

1.   Day: AWMF Batch 0 - 9   (8988 pages)
2.   Day: AWMF Batch 10 - 19 (9368 pages)
3.   Day: AWMF Batch 20 - 29 (8386 pages)
4.   Day: AWMF Batch 30 - 39 (8765 pages)
5.   Day: AWMF Batch 40 - 49 (8584 pages)
6.   Day: AWMF Batch 50 - 59 (9458 pages)
7.   Day: AWMF Batch 60 - 67 (6908 pages)

If done, enter a check mark e.g. [✓] behind the day, to keep track

In [7]:
# SET TO CURRENT DAY:
day = 1

# Sample data: lists of API keys and folder names
api_keys_dict = { 0: 'llx-QUF47pz0VJGzUrdUzFpOgmjHpH9TpAmudTAiJYmF5aCyPrgN',
                  1: 'llx-u4t0fZSHsXQAhit7M7Hjf1MJTX3BoxAKJW9jswGTfwKf9tFB',
                  2: 'llx-RSBs3YYka7OpvtSu33s24LKe9HzmQvzzJ0TidvxouF6TQwrz',
                  3: 'llx-LoEWhKswtx0ltVIeuldBGKxykNfDLb8WpfJ0rQiqGVYzgFge',
                  4: 'llx-9zwZrC1cPhoNSxhfDlzQc2OuEe9b73WJOmXtAra9kD1HxVRF',
                  5: 'llx-AHsZMxFAh7wjO8IWCEcU70QWobBE1h9co7uQzr7CRpXhcrrN', # timstrohmeyer@t-online.com
                  6: 'llx-jAfQU0bh39DQLllFndmkhEgrxbuv5BdifGDSDnhyLPhZvbd9', # timstrohmeyer6@gmail.com
                  7: 'llx-BI8SYu3Wpy3nW5XWydwaRES9mVEM8VWUCqgNwSRyxQODchUv', # tim.strohmeyer@tum.de
                  8: 'llx-hZjoji4yRKNqR286LvpM3zSZCTPfB2SkKfRKap1qCQU3dEYO', # timstrohmeyer24@gmail.com
                  9: 'llx-FQYdSjuA4KVfXvDLs89EMEE50s7QvNDsmmQze6nJS9eQS4fV'} # timstrohmeyer3@gmail.com


batch_dir_to_parse = {1: [i for i in range(0,10)],
                      2: [i for i in range(10,20)],
                      3: [i for i in range(20,30)],
                      4: [i for i in range(30,40)],
                      5: [i for i in range(40,50)],
                      6: [i for i in range(50,60)],
                      7: [i for i in range(60,68)]}

assignment_dict = {}

batches_to_parse = batch_dir_to_parse[day]

for idx in batches_to_parse:
    # Assign API key to each document index
    assignment_dict["AWMF_batch_"+str(idx)] = api_keys_dict[idx % len(api_keys_dict)]

print(f"batch_dir_to_parse: {batches_to_parse}")
print(assignment_dict)

batch_dir_to_parse: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
{'AWMF_batch_0': 'llx-QUF47pz0VJGzUrdUzFpOgmjHpH9TpAmudTAiJYmF5aCyPrgN', 'AWMF_batch_1': 'llx-u4t0fZSHsXQAhit7M7Hjf1MJTX3BoxAKJW9jswGTfwKf9tFB', 'AWMF_batch_2': 'llx-RSBs3YYka7OpvtSu33s24LKe9HzmQvzzJ0TidvxouF6TQwrz', 'AWMF_batch_3': 'llx-LoEWhKswtx0ltVIeuldBGKxykNfDLb8WpfJ0rQiqGVYzgFge', 'AWMF_batch_4': 'llx-9zwZrC1cPhoNSxhfDlzQc2OuEe9b73WJOmXtAra9kD1HxVRF', 'AWMF_batch_5': 'llx-AHsZMxFAh7wjO8IWCEcU70QWobBE1h9co7uQzr7CRpXhcrrN', 'AWMF_batch_6': 'llx-jAfQU0bh39DQLllFndmkhEgrxbuv5BdifGDSDnhyLPhZvbd9', 'AWMF_batch_7': 'llx-BI8SYu3Wpy3nW5XWydwaRES9mVEM8VWUCqgNwSRyxQODchUv', 'AWMF_batch_8': 'llx-hZjoji4yRKNqR286LvpM3zSZCTPfB2SkKfRKap1qCQU3dEYO', 'AWMF_batch_9': 'llx-FQYdSjuA4KVfXvDLs89EMEE50s7QvNDsmmQze6nJS9eQS4fV'}


In [8]:
# Set the folder and API Key Index
index = 1

In [13]:
import nest_asyncio
nest_asyncio.apply()

# Set the current AWMF_Batch index you are working with:
'''
Remember:

1. Day: 0 - 9
2. Day: 10 - 19
3. Day: 20 - 29
4. Day: 30 - 39
5. Day: 40 - 49
6. Day: 50 - 59
7. Day: 60 - 67

'''

LLAMA_CLOUD_API_KEY = assignment_dict["AWMF_batch_" + str(index)]
print(LLAMA_CLOUD_API_KEY)

parser = LlamaParse(
    api_key=LLAMA_CLOUD_API_KEY,
    result_type="markdown"  # "markdown" and "text" are available
    #verbose=True
)

def get_meta(file_path):
    return {"source": file_path}

directory_path = "/Users/timstrohmeyer/Downloads/AWMF_Database_edited/AWMF_batch_" + str(index) # test

file_extractor = {".pdf": parser}
reader = SimpleDirectoryReader(input_dir=directory_path,
                                #recursive=True,
                                num_files_limit=2, # comment this when doing real ingestion
                                file_extractor=file_extractor,
                                filename_as_id=True,
                                file_metadata=get_meta
                                )
docs = reader.load_data()

llx-u4t0fZSHsXQAhit7M7Hjf1MJTX3BoxAKJW9jswGTfwKf9tFB
Started parsing the file under job_id cac11eca-baf9-4f5f-8a7b-c75661f07eb2
Started parsing the file under job_id cac11eca-8015-40fe-8239-d7e6da5da86e


In [14]:
for i in range(len(docs)):
    print(i)
    print(docs[i].metadata['source'])

0
/Users/timstrohmeyer/Downloads/AWMF_Database_edited/AWMF_batch_1/001-021l_S3_Invasive_Beatmung_2017-12.pdf
1
/Users/timstrohmeyer/Downloads/AWMF_Database_edited/AWMF_batch_1/001-022l_S1_Regionalanaesthesie-ambulante-Patienten-Empfehlung-zur-Durchfuehrung_2021-03.pdf


### 2. **Text Preprocessing**
#### 2.1 **Text Normalization**
preprocess the textual data by getting rid of points in shortcut notations such as "bzw.,z.B., d.h." etc

In [15]:
# Normalize Text
import re
# List of expressions to replace
expressions_to_replace = [
    "bzw\.", "z.B\.", "med\.", "Med\.", "Dr\.", "zit\.", "n\.", "e.V\.", "rer\.", "nat\.", "Prof\.", "B.A\.",
    "Ca\.", "ca\.", "usw\.", "v\. a\.", "p\. p\.", "s\. o\.", "s\. u\.", "sog\.", "u\. a\.", "vs\.",
    "Min\.", "et al\.", "ärztl\.", "evtl\.", "ggf\.", "d\.h\.", "Abb\.", "u\.a\.", "usw\.", "etc\.", "vgl\."
]


for i in range(len(docs)):
    print("iteration:",i)
    # Replace the expressions in the markdown text
    for expression in expressions_to_replace:
        docs[i].text = re.sub(expression, expression.replace("\\.", ""), docs[i].text)

iteration: 0
iteration: 1


### 3. Chunking Algorithm

In [17]:
'''
Method:
1. Markdown Chunking: First use the MarkdownTextSplitter to do “rule-based chunking” using the titles and headers.
2. Semantic Chunking:
    2.1. Check the chunk size of the resulting chunks
    2.2. If chunk size is below min token threshold (i.e. 20 tokens) --> merge the chunk with the previous chunk
    2.3. If chunk size exceeds the max chunk size of the embedding model (i.e. 512 for e5-multilingual) --> subchunk that chunk with the SemanticChunker.
        2.3.1. Find the chunks with token size above max token limit (512 tokens)
        2.3.2. Split the chunks into subchunks using breakpoint value of 95 (to begin with)
        2.3.3. Check subchunks:
            2.3.3.1 if the created subchunks are smaller than min token limit --> merge with previous subchunk
            2.3.3.2 Substitute the original "parent chunk" with its subchunks (Note: Metadata of parent chunks are copied to children)
        4. Iteratively check again the size each resulting subchunk. (go back to 2.3.1)
        5. If the subchunks cannot be broken down any further, (i.e. SemanticChunker is unable to break at threshold 95) --> reduce threshold to 85
'''

# MarkdownTextSplitter
from langchain.text_splitter import MarkdownHeaderTextSplitter
import tiktoken
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings
from tqdm import tqdm
import re
import json
import unicodedata

# helper functions

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    #num_tokens = len(encoding.encode(string, disallowed_special = set()))

    return num_tokens

def semantic_chunker(threshold_amount):
    """Creates an instance of a SemanticChunker. Input threshold value for breakpoint splitting"""
    text_splitter = SemanticChunker(
        OpenAIEmbeddings(model = "text-embedding-3-small"),
        breakpoint_threshold_type="percentile",
        #breakpoint_threshold_type="standard_deviation"
        #breakpoint_threshold_type="interquartile"
        breakpoint_threshold_amount = threshold_amount
    )
    return text_splitter

def print_all_chunks(list_of_docs,with_page_content=False, with_num_tokens=True):

    for i in range(len(list_of_docs)):
        num_of_tokens = num_tokens_from_string(list_of_docs[i].page_content, "cl100k_base")
        if with_num_tokens == True:
            print(f"num. of tokens in chunk {i} is: {num_of_tokens}")
        if with_page_content == True:
            #print(f"-----------------------------CHUNK {i} ----------------------------------")
            #print(list_of_docs[i].page_content)
            print(f"---")
            print(list_of_docs[i].page_content)

def find_idxs_below_min_chunk_size(list_of_docs, min_chunk_size=20, show_indices=True):

    idxs_below_min_chunk_size = []

    for i in range(len(list_of_docs)):
        num_of_tokens = num_tokens_from_string(list_of_docs[i].page_content, "cl100k_base")

        # if chunk size is below min_chunk_size, append index to list
        if num_of_tokens < min_chunk_size:
            idxs_below_min_chunk_size.append(i)

    num_of_idxs_in_list = len(idxs_below_min_chunk_size)

    if show_indices==True:
        print(f"{num_of_idxs_in_list} indices below min threshold: {idxs_below_min_chunk_size}")

    return idxs_below_min_chunk_size

def find_idxs_above_max_chunk_size(list_of_docs, max_chunk_size=512):

    idxs_above_max_chunk_size = []

    for i in range(len(list_of_docs)):
        num_of_tokens = num_tokens_from_string(list_of_docs[i].page_content, "cl100k_base")

        # if chunk size is below min_chunk_size, append index to list
        if num_of_tokens > max_chunk_size:
            idxs_above_max_chunk_size.append(i)

    num_of_idxs_in_list = len(idxs_above_max_chunk_size)

    print(f"{num_of_idxs_in_list} indices above max threshold: {idxs_above_max_chunk_size}")

    return idxs_above_max_chunk_size

def remove_below_min_chunks_list(list_of_docs, list_blw_min_idx, min_chunk_size=20):

    # inverse list order to not get indexing problems
    sorted_max_indices = sorted(list_blw_min_idx, reverse=True)
    #print(sorted_max_indices)

    for m in sorted_max_indices:

        num_of_tokens = num_tokens_from_string(list_of_docs[m].page_content, "cl100k_base")

        if num_of_tokens < min_chunk_size and m != 0:
            update_data = {
                "page_content": list_of_docs[m-1].page_content + " \n" + list_of_docs[m].page_content,
            }
            new_doc_node = list_of_docs[m-1].copy(update = update_data)

            # delete the two old nodes and subsitute it with the new node
            list_of_docs.pop(m)
            list_of_docs.pop(m-1)

            #insert new merged node into list
            list_of_docs.insert(m-1,new_doc_node)

def list_of_str_2_list_of_docs(list_of_str, doc_obj):

    list_of_docs = []

    for n in range(len(list_of_str)):

        data = {
            "page_content": list_of_str[n]
        }
        new_doc_obj = doc_obj.copy(update = data)
        list_of_docs.append(new_doc_obj)

    #print(list_of_docs)
    return list_of_docs

def remove_above_max_chunks_list(list_of_docs, list_abv_max_idx, breakpoint_thresh_value=95):

    # inverse list order to not get indexing problems
    sorted_max_indices = sorted(list_abv_max_idx, reverse=True)
    #print(sorted_max_indices)

    for m in tqdm(sorted_max_indices):

            # semantically split chunk
            subchunks = semantic_chunker(breakpoint_thresh_value).split_text(list_of_docs[m].page_content)

            # right now subchunks is a list of str (i.e. each element only contains text).
            # convert this list of strings into a list of docs
            node = list_of_docs[m].copy() # create copy of parent node (needed for list_of_str_2_list_of_docs fct.)
            docs = list_of_str_2_list_of_docs(subchunks, node)

            # Check, that the created subchunks are of token size > min_chunk_size (=20) --> if not, merge them with previous subchunk
            subchunk_idxs_below_min_thresh = find_idxs_below_min_chunk_size(docs, min_chunk_size=100, show_indices=False)
            remove_below_min_chunks_list(docs, subchunk_idxs_below_min_thresh, min_chunk_size=100)

            num_of_subchunks=len(docs)

            #create copy of the node
            copy_node_2 = list_of_docs[m].copy()
            #print(copy_node)

            # delete node from list
            list_of_docs.pop(m)

            for n in reversed(range(num_of_subchunks)):

                update_data = {
                    "page_content": docs[n].page_content
                }
                copy_node_2_updated = copy_node_2.copy(update = update_data)

                #insert new merged node into list
                list_of_docs.insert(m, copy_node_2_updated)

def write_results_in_txt(list_of_docs,leitlinien_doc,list_of_above_max_indices, output_file):
    with open(output_file, 'w', encoding='utf-8') as file:
        file.write(f"-----------------------------RESULTS ----------------------------------\n")
        file.write(f"Leitlinien doc: {leitlinien_doc} \n")
        file.write(f"final num. of chunks: {len(list_of_docs)}\n")
        file.write(f"chunks over max. token limit: {list_of_above_max_indices}\n")

        for i in range(len(list_of_docs)):
            num_of_tokens = num_tokens_from_string(list_of_docs[i].page_content, "cl100k_base")
            file.write(f"num. of tokens in chunk {i} is: {num_of_tokens} \n")
            file.write(f"-----------------------------CHUNK {i} ----------------------------------\n")
            file.write(list_of_docs[i].page_content + '\n')

# Page number functions
'''
Method: "Unsexy but works"
1. Read Markdown Text. Whenever the page delimiter (i.e. "---") is encountered, write the current page count above the line (e.g. (Page x)) (Note: Llamaparse reliably detects all page breaks)
2. For each Header (marked by ###,##,#) write page number into the Header.
3. Let MarkdownHeaderTextSplitter split the text (During Chunking). This will automatically create chunks with metadata containing "Header" information.
4. Now we extract the Page number previously written into the Header, and write it as a new entry in the metadata dictionary under "Page Number"
5. Delete all the (Page x) added previously to the headers and page_content to restore our original markdown text.

'''
def add_last_page_delimiter(markdown_text):
    '''
    Add a --- delimiter for the last page
    '''
    # Ensure the markdown text ends with a newline before adding the delimiter
    if not markdown_text.endswith('\n'):
        markdown_text += '\n'

    # Add the --- delimiter at the end
    updated_markdown_text = markdown_text + '---\n'

    return updated_markdown_text

def count_and_update_delimiters(markdown_text):
    # Compile regex pattern to match "---" delimiters on separate lines
    delimiter_pattern = re.compile(r'^\s*---\s*$', re.MULTILINE)

    # Initialize count
    delimiter_count = 0

    # Function to replace each "---" delimiter with "--- {count} delimiter(s) found"
    def replace(match):
        nonlocal delimiter_count
        delimiter_count += 1
        return f'\n(Page {delimiter_count})\n---\n'

    # Replace each "---" delimiter
    updated_markdown_text = re.sub(delimiter_pattern, replace, markdown_text)

    return updated_markdown_text

def modify_headers_with_page_numbers(markdown_text):
    # Compile regex pattern to match headers (###, ##, #) and "page_nr: x"
    header_pattern = re.compile(r'^(#+)\s+(.*)$', re.MULTILINE)
    page_number_pattern = re.compile(r'(Page \s*(\d+))', re.IGNORECASE)

    # Function to replace each header with header + page number
    def replace(match):
        header_level = match.group(1)
        header_text = match.group(2)
        page_nr_match = page_number_pattern.search(markdown_text, match.end())
        if page_nr_match:
            page_nr = page_nr_match.group(1)
            return f'{header_level} {header_text} ({page_nr})'
        else:
            return match.group(0)

    # Replace each header
    updated_markdown_text = re.sub(header_pattern, replace, markdown_text)

    return updated_markdown_text

def update_metadata_with_page_numbers(md_header_splits):
    '''
    This function:
    1. reads the page numbers, previously inserted into the headers, and writes them into a seperate metadata entry: Page Number
    2. deletes all the page numbers inserted previously into the header metadata entries
    '''
    # Compile regex pattern to extract page numbers from headers
    page_number_pattern = re.compile(r'\(Page\s+(\d+)\)')

    for header_split in md_header_splits:
        metadata = header_split.metadata
        page_content = header_split.page_content
        page_numbers = []

        if metadata:
            last_header_value = list(metadata.values())[-1]
            page_number_match = page_number_pattern.search(last_header_value)
            if page_number_match:
                page_number = int(page_number_match.group(1))
                page_numbers.append(page_number)
                metadata['Page Number'] = page_numbers
            for key, value in metadata.items():
                if re.search(r'\bHeader\b', key):
                  # Remove the page number and surrounding brackets from the header
                  value_without_page_number = re.sub(page_number_pattern, '', value).strip()

                  # Update the metadata dictionary
                  metadata[key] = value_without_page_number
                if key == 'Page Number':
                    page_nums = set(int(match.group(1)) for match in page_number_pattern.finditer(page_content))

                    # If page_numbers is None, set it to [1]
                    if page_numbers is None:
                        page_numbers = [1]
                    for num in page_nums:
                        if num not in page_numbers:
                            page_numbers.append(num)
                            metadata['Page Number'] = page_numbers

def update_page_content(md_header_splits):
    '''
    This function deletes all the page numbers (i.e. (Page x)) inserted previously into the page content
    '''
    # Compile regex pattern to match (Page x)
    page_number_pattern = re.compile(r'\(Page\s+\d+\)')

    for header_split in md_header_splits:
        page_content = header_split.page_content
        # Remove (Page x) from the page content
        updated_page_content = re.sub(page_number_pattern, '', page_content)
        # Update the page_content in md_header_splits
        header_split.page_content = updated_page_content

def update_metadata_with_source(md_header_splits,guidline_metadata):
    '''
    This function includes the document source into the final chunks metadata
    '''
    for header_split in md_header_splits:
        guideline_name = guidline_metadata['Guideline_Name']

        if guideline_name.endswith('.pdf'):
          guideline_name = guideline_name[:-4]

        header_split.metadata['source'] = guideline_name

def update_metadata_with_validity(md_header_splits, guidline_metadata):
    '''
    This function includes the document source into the final chunks metadata
    '''
    for header_split in md_header_splits:
        # split.metadata['Page Number'][0] +=1
        if "abgelaufen" not in guidline_metadata['Guideline_Name']:
            header_split.metadata["Gültigkeit"] = "Gültig"
        else:
            header_split.metadata["Gültigkeit"] = "Abgelaufen"

def update_metadata_with_Fachgesellschaft(md_header_splits, guidline_metadata):
    '''
    This function includes the Fachgesellschaft into the final chunks metadata
    '''
    for header_split in md_header_splits:
        header_split.metadata['Fachgesellschaft'] = guidline_metadata['Fachgesellschaft']

def update_metadata_with_href(md_header_splits, guidline_metadata):
    '''
    This function includes the download_href into the final chunks metadata
    '''
    for header_split in md_header_splits:
        header_split.metadata['href'] = guidline_metadata['download_href']

def normalize_unicode(input_string):
    return unicodedata.normalize('NFC', input_string)

# 1. MarkdownTextSplitter

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, strip_headers=False)

# Settings
min_chunk_size = 300
max_chunk_size = 1000 #ada: 8191, e-5: 512

file_path = '/Users/timstrohmeyer/Downloads/Database_Metadata/guideline_metadata.json'
with open(file_path, 'r', encoding='utf-8') as file:
    json_metadata = json.load(file)

# List containing all chunks
doc_chunks = []

d = 0
for d in range(len(docs)):

    documents = docs[d]
    # source_name = documents.metadata['source'].split('/')[-1]
    source_name = normalize_unicode(documents.metadata['source'].split('/')[-1])
    print("source_name:",source_name)
    guidline_metadata = json_metadata[source_name]
    print("guidline_metadata:",guidline_metadata)

    # add page number data into markdown
    updated_text = add_last_page_delimiter(documents.text)
    updated_text = count_and_update_delimiters(updated_text)
    modified_text = modify_headers_with_page_numbers(updated_text)

    # Perform Markdown Text Splitting
    md_header_splits = markdown_splitter.split_text(modified_text)
    #md_header_splits = markdown_splitter.split_text(documents[0].text)

    # add page metadata
    update_metadata_with_page_numbers(md_header_splits)
    update_metadata_with_source(md_header_splits, guidline_metadata)
    update_metadata_with_validity(md_header_splits, guidline_metadata)
    update_metadata_with_Fachgesellschaft(md_header_splits, guidline_metadata)
    update_metadata_with_href(md_header_splits, guidline_metadata)
    #update_page_content(md_header_splits)

    print("----------------------------------MarkdownHeaderTextSplitter----------------------------------")

    print(f"num. of chunks after MarkdownHeaderTextSplitter: {len(md_header_splits)}")

    for i in range(len(md_header_splits)):
        str_ = num_tokens_from_string(md_header_splits[i].page_content, "cl100k_base")
        print(f"num. of tokens in chunk {i} is: {str_}")
        #print(md_header_splits[i].metadata)

    print("Indices that are below min/above max token size: ")
    list_of_below_min_indices = find_idxs_below_min_chunk_size(md_header_splits, min_chunk_size)
    list_of_above_max_indices= find_idxs_above_max_chunk_size(md_header_splits, max_chunk_size)
    i = 0
    #num_of_idxs_in_list = []
    previous_indices = None
    breakpoint_thresh_value = 95

    print("----------------------------------MIN_MAX_SUBCHUNKING:----------------------------------")
    while len(list_of_below_min_indices) > 0 or len(list_of_above_max_indices) > 0:

        list_of_below_min_indices = find_idxs_below_min_chunk_size(md_header_splits, min_chunk_size)

        if len(list_of_below_min_indices) > 0:
            remove_below_min_chunks_list(md_header_splits, list_of_below_min_indices, min_chunk_size)
            print(f"-----------------------CHUNKS AFTER MIN REMOVAL {i}--------------------------")
            print(f"num. of chunks after {i} MIN_SUBCHUNKING: {len(md_header_splits)}")
            #print_all_chunks(md_header_splits, with_page_content=False)

        list_of_above_max_indices= find_idxs_above_max_chunk_size(md_header_splits, max_chunk_size)

        # Check, if two consecutive numbers in the num_of_idxs_in_list = [] are the same, decrease the breakpoint threshold by 10.
        if previous_indices is not None and list_of_above_max_indices == previous_indices and breakpoint_thresh_value!=75:
            breakpoint_thresh_value=breakpoint_thresh_value-10

        if list_of_above_max_indices == previous_indices and breakpoint_thresh_value==75:
            print("Finished!")
            break

        # Update the previous indices for the next iteration
        previous_indices = list_of_above_max_indices

        if len(list_of_above_max_indices) > 0:
            remove_above_max_chunks_list(md_header_splits, list_of_above_max_indices, breakpoint_thresh_value)
            print(f"-----------------------CHUNKS AFTER MAX REMOVAL {i}--------------------------")
            print(f"num. of chunks after {i} MAX_SUBCHUNKING: {len(md_header_splits)}")
            #print_all_chunks(md_header_splits, with_page_content=False)

        #print(md_header_splits)
        #print(len(md_header_splits))

        i = i+1

    print("-----------------------FINAL CHUNKS--------------------------")

    doc_chunks.append(md_header_splits)
    print_all_chunks(md_header_splits, with_page_content=False)

    # leitlinien_doc= documents.id_
    # output_file = f"chunking_results_{d}.txt"
    # open(output_file, 'w').close()
    # write_results_in_txt(md_header_splits,leitlinien_doc,list_of_above_max_indices, output_file)

source_name: 001-021l_S3_Invasive_Beatmung_2017-12.pdf
guidline_metadata: {'Fachgesellschaft': ['Anästhesiologie & Intensivmedizin'], 'download_href': 'https://register.awmf.org/assets/guidelines/001-021l_S3_Invasive_Beatmung_2017-12.pdf', 'Guideline_Name': '001-021l_S3_Invasive_Beatmung_2017-12.pdf'}
----------------------------------MarkdownHeaderTextSplitter----------------------------------
num. of chunks after MarkdownHeaderTextSplitter: 475
num. of tokens in chunk 0 is: 614
num. of tokens in chunk 1 is: 726
num. of tokens in chunk 2 is: 54
num. of tokens in chunk 3 is: 161
num. of tokens in chunk 4 is: 752
num. of tokens in chunk 5 is: 54
num. of tokens in chunk 6 is: 888
num. of tokens in chunk 7 is: 18
num. of tokens in chunk 8 is: 13
num. of tokens in chunk 9 is: 17
num. of tokens in chunk 10 is: 23
num. of tokens in chunk 11 is: 23
num. of tokens in chunk 12 is: 18
num. of tokens in chunk 13 is: 20
num. of tokens in chunk 14 is: 17
num. of tokens in chunk 15 is: 30
num. of to

100%|██████████| 61/61 [01:56<00:00,  1.91s/it]


-----------------------CHUNKS AFTER MAX REMOVAL 0--------------------------
num. of chunks after 0 MAX_SUBCHUNKING: 343
25 indices below min threshold: [50, 54, 57, 63, 76, 81, 88, 91, 92, 106, 111, 135, 149, 167, 170, 177, 179, 263, 265, 266, 268, 279, 301, 320, 334]
-----------------------CHUNKS AFTER MIN REMOVAL 1--------------------------
num. of chunks after 1 MIN_SUBCHUNKING: 320
34 indices above max threshold: [3, 41, 49, 52, 55, 71, 75, 76, 81, 83, 89, 98, 99, 101, 102, 104, 112, 120, 124, 137, 154, 156, 163, 164, 169, 241, 246, 247, 249, 281, 292, 298, 311, 312]


100%|██████████| 34/34 [02:21<00:00,  4.17s/it]


-----------------------CHUNKS AFTER MAX REMOVAL 1--------------------------
num. of chunks after 1 MAX_SUBCHUNKING: 342
15 indices below min threshold: [50, 54, 75, 81, 110, 113, 136, 150, 168, 170, 262, 263, 267, 318, 332]
-----------------------CHUNKS AFTER MIN REMOVAL 2--------------------------
num. of chunks after 2 MIN_SUBCHUNKING: 328
26 indices above max threshold: [3, 41, 49, 52, 72, 93, 103, 104, 106, 107, 109, 117, 125, 129, 142, 159, 161, 168, 169, 174, 246, 251, 255, 299, 305, 318]


...

-----------------------CHUNKS AFTER MAX REMOVAL 3--------------------------
num. of chunks after 3 MAX_SUBCHUNKING: 16
4 indices below min threshold: [7, 8, 9, 13]
-----------------------CHUNKS AFTER MIN REMOVAL 4--------------------------
num. of chunks after 4 MIN_SUBCHUNKING: 13
3 indices above max threshold: [5, 8, 10]


100%|██████████| 3/3 [00:03<00:00,  1.03s/it]


-----------------------CHUNKS AFTER MAX REMOVAL 4--------------------------
num. of chunks after 4 MAX_SUBCHUNKING: 15
2 indices below min threshold: [8, 12]
-----------------------CHUNKS AFTER MIN REMOVAL 5--------------------------
num. of chunks after 5 MIN_SUBCHUNKING: 13
2 indices above max threshold: [5, 10]


100%|██████████| 2/2 [00:01<00:00,  1.07it/s]

-----------------------CHUNKS AFTER MAX REMOVAL 5--------------------------
num. of chunks after 5 MAX_SUBCHUNKING: 14
1 indices below min threshold: [11]
-----------------------CHUNKS AFTER MIN REMOVAL 6--------------------------
num. of chunks after 6 MIN_SUBCHUNKING: 13
2 indices above max threshold: [5, 10]
Finished!
-----------------------FINAL CHUNKS--------------------------
num. of tokens in chunk 0 is: 372
num. of tokens in chunk 1 is: 462
num. of tokens in chunk 2 is: 315
num. of tokens in chunk 3 is: 891
num. of tokens in chunk 4 is: 991
num. of tokens in chunk 5 is: 1441
num. of tokens in chunk 6 is: 963
num. of tokens in chunk 7 is: 719
num. of tokens in chunk 8 is: 806
num. of tokens in chunk 9 is: 544
num. of tokens in chunk 10 is: 1207
num. of tokens in chunk 11 is: 383
num. of tokens in chunk 12 is: 435


In [18]:
for i in range(len(doc_chunks)):
  #print(f"doc name: {doc_chunks[i]}")
  print("New doc")
  for j in range(len(doc_chunks[i])):
    print(doc_chunks[i][j].metadata)
    # print(doc_chunks[i][j].metadata['Page Number'])

New doc
{'Header 1': 'S3-Leitlinie Invasive Beatmung und Einsatz extrakorporaler Verfahren bei akuter respiratorischer Insuffizienz', 'Page Number': [1], 'source': '001-021l_S3_Invasive_Beatmung_2017-12', 'Gültigkeit': 'Gültig', 'Fachgesellschaft': ['Anästhesiologie & Intensivmedizin'], 'href': 'https://register.awmf.org/assets/guidelines/001-021l_S3_Invasive_Beatmung_2017-12.pdf'}
{'Header 1': 'S3-Leitlinie Invasive Beatmung und Einsatz extrakorporaler Verfahren bei akuter respiratorischer Insuffizienz', 'Page Number': [2], 'source': '001-021l_S3_Invasive_Beatmung_2017-12', 'Gültigkeit': 'Gültig', 'Fachgesellschaft': ['Anästhesiologie & Intensivmedizin'], 'href': 'https://register.awmf.org/assets/guidelines/001-021l_S3_Invasive_Beatmung_2017-12.pdf'}
{'Header 1': 'Beteiligte Fachgesellschaften/Organisationen (Mandatsträger):', 'Page Number': [3], 'source': '001-021l_S3_Invasive_Beatmung_2017-12', 'Gültigkeit': 'Gültig', 'Fachgesellschaft': ['Anästhesiologie & Intensivmedizin'], 'href'

### 4. Write Chunks to txt file

In [ ]:
import os

def write_chunks_to_file(md_header_splits, directory_path, file_name):

    # Create the directory if it doesn't exist
    os.makedirs(directory_path, exist_ok=True)

    # Construct the full file path
    file_path = os.path.join(directory_path, file_name)


    with open(file_path, 'w') as file:
        file.write('     ***CHUNK***\n')
        for i in range(len(md_header_splits)):
            file.write('\n')
            file.write(md_header_splits[i].page_content)  # Write the indented text from the list and remove leading/trailing whitespace
            file.write('\n\n')
            file.write('     ***CHUNK***\n')
            #file.write('\n')  # Write "***" after each text element

# Specify the base path where you want to create the directories
base_path = '/content/gdrive/MyDrive/chunk_results'  # Change this to your desired path in Google Drive
dir_name = f'AWMF_batch_{index}'
dir_path = os.path.join(base_path, dir_name)
os.makedirs(dir_path, exist_ok=True)

# Create directories iteratively
for i in range(len(doc_chunks)):
    filename = doc_chunks[i][0].metadata["source"].rstrip('.pdf') + '.txt'
    #filename = f"render_ready_chunk_results_{i}.txt"
    write_chunks_to_file(doc_chunks[i], dir_path, filename)
    print(f"{filename} created")

### 5. Pincone Loading

In [20]:
# Preprocessing Data Load for splits

splits = doc_chunks
splits_metadata_source = [[] for _ in range(len(splits))]
splits_metadata_page = [[] for _ in range(len(splits))]
splits_metadata_Gültigkeit = [[] for _ in range(len(splits))]
splits_page_content = [[] for _ in range(len(splits))]
splits_metadata_Fachgesellschaft = [[] for _ in range(len(splits))]
splits_metadata_Datenbank = [[] for _ in range(len(splits))]
previous_page_number = 0  # Initialize previous page number

#preprocess
for i in range(len(splits)):
    for j in range(len(splits[i])):
        splits_page_content[i].append(splits[i][j].page_content)
        metadata = splits[i][j].metadata
        splits_metadata_source[i].append(metadata['source'])
        splits_metadata_Gültigkeit[i].append(metadata['Gültigkeit'])
        splits_metadata_Fachgesellschaft[i].append(metadata['Fachgesellschaft'])
        # Extract page number
        if 'Page Number' in metadata:
            current_page_number = [str(num) for num in metadata['Page Number']]

        if 'Page Number' in metadata:  # Check if 'Page Number' exists
            current_page_number = [str(num) for num in metadata['Page Number']]
            previous_page_number = current_page_number
        else:  # If 'Page Number' doesn't exist
            current_page_number = previous_page_number + 1  # Calculate page number

        splits_metadata_page[i].append(current_page_number)
        splits_metadata_Datenbank[i].append("AWMF")


In [ ]:
# Creating new index
from pinecone import Pinecone, ServerlessSpec
from openai import OpenAI
#model = SentenceTransformer("BAAI/bge-small-en-v1.5") # 'all-MiniLM-L6-v2'
#model = OpenAIEmbeddings() # 'all-MiniLM-L6-v2'

client = OpenAI()  # get API key from platform.openai.com

pc = Pinecone(api_key=PINECONE_API_KEY)

#INDEX_NAME = 'leitliniengpt-vdb'
INDEX_NAME = 'leitliniengpt-vdb'

if INDEX_NAME in [index.name for index in pc.list_indexes()]:
    pc.delete_index(INDEX_NAME)

pc.create_index(
    name=INDEX_NAME,
    dimension=1536,
    metric='cosine',
    spec=ServerlessSpec(
        cloud='aws',
        region='eu-west-1'
    )
)

In [ ]:
index = pc.Index(INDEX_NAME)
index.describe_index_stats()

In [ ]:
# Upserting Data to pinecone
batch_size=5

# id_file_path = "/content/gdrive/MyDrive/Colab Notebooks/last_used_id.txt"

# Select Model
MODEL = "text-embedding-3-small"

def convert_to_ascii(input_string):
    replacements = {
        'Ä': 'A', 'Ö': 'O', 'Ü': 'U', 'ä': 'a', 'ö': 'o', 'ü': 'u', 'ß': 'ss', 'é': 'e', 'á': 'a',
        'à': 'a', 'è': 'e', 'ê': 'e', 'ë': 'e', 'î': 'i', 'ï': 'i', 'ô': 'o', 'û': 'u', 'ù': 'u',
        'ç': 'c', 'ñ': 'n', 'ã': 'a', 'õ': 'o', 'Á': 'A', 'É': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U',
        'Ñ': 'N', 'ã': 'a', 'õ': 'o'
    }
    for non_ascii, ascii_equiv in replacements.items():
        input_string = input_string.replace(non_ascii, ascii_equiv)
    return input_string

total_id = 0


for doc_num in range(len(splits)):
    # Initialize ID counter
    current_id = 0
    for i in tqdm(range(0, len(splits[doc_num]), batch_size)):
        # find end of batch
        i_end = min(i+batch_size, len(splits[doc_num]))

        # create IDs batch with increasing numbers
        ids = [str(current_id + x) for x in range(i_end - i)]

        # Increment ID counter
        current_id += i_end - i
        total_id += i_end - i

        # create metadata batch
        # metadatas = [
        #                 {'text': text,'Source': source, 'Page': page, 'Gültigkeit': gültigkeit, 'Fachgesellschaft': Fachgesellschaft, 'Datenbank': Datenbank}
        #                 for text, source, page, gültigkeit, Fachgesellschaft, Datenbank in zip(splits_page_content[doc_num][i:i_end], splits_metadata_source[doc_num][i:i_end], splits_metadata_page[doc_num][i:i_end], splits_metadata_Gültigkeit[doc_num][i:i_end], splits_metadata_Fachgesellschaft[doc_num][i:i_end],splits_metadata_Datenbank[doc_num][i:i_end])
        #             ]

        metadatas = [
                        {'text': text, 'Source': source, 'Page': page, 'Gültigkeit': gültigkeit, 'Fachgesellschaft': Fachgesellschaft, 'Datenbank': Datenbank}
                          for text, source, page, gültigkeit, Fachgesellschaft, Datenbank in zip(
                              splits_page_content[doc_num][i:i_end],
                              splits_metadata_source[doc_num][i:i_end],
                              splits_metadata_page[doc_num][i:i_end],
                              splits_metadata_Gültigkeit[doc_num][i:i_end],
                              splits_metadata_Fachgesellschaft[doc_num][i:i_end],
                              splits_metadata_Datenbank[doc_num][i:i_end]
                          )
                      ]

        # Combine ids with corresponding source in the specified format
        combined_id = [f"Chunk_{id}_{convert_to_ascii(metadata['Source'])}" for id, metadata in zip(ids, metadatas)]
        print("combined_ids:",combined_id)

        # create embeddings
        lines_batch = splits_page_content[doc_num][i: i+batch_size]
        res = client.embeddings.create(input=lines_batch, model=MODEL)
        embeds = [record.embedding for record in res.data]

        # create records list for upsert
        records = zip(combined_id, embeds, metadatas)
        # upsert to Pinecone
        index.upsert(vectors=records)
    print(f"Doc {doc_num} with {i_end} chunks upserted!")
    print(f"Total upserted: {total_id}")

# # Write the last used ID to the file
# with open(id_file_path, "w") as id_file:
#     id_file.write(str(current_id))